Dyna Q model, CVAE, reward model and terminal model trained

In [8]:
#this allows relative imports in notebook
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
name='Experiment1'

In [10]:
from UCLSE.dyna_q.Experiment1a import Experiment
from UCLSE.dyna_q.dyna_q import TabularMemory
from UCLSE.dyna_q.benchmarking import BenchmarkAgent, SpoofAgent, DoNothing
import numpy as np
import pandas as pd
import os
import torch
import visdom
import matplotlib.pyplot as plt

In [11]:
vis=visdom.Visdom(port=8097)

In [12]:
def reward_oracle(observation,cutoff=50,ub=6,lb=-2,lamb=0.5):


    distance=observation.distance
    inventory=observation.inventory
    orders_out=observation.orders_out
    bid_change=observation.bid_change
    bid_ask_spread=observation.bid_ask_spread
    time_left=observation.time_left

    ans=lamb*bid_change

    if inventory==0:   #terminal            
            
            ans+=-(1-lamb)*distance
            ans-=lamb*bid_change

    elif inventory>1: #terminal
            
            ans+=-bid_ask_spread*(inventory-1)
            ans+=-(1-lamb)*distance
            ans+=-1 #penalty
           
    else:

            if orders_out>0: 
                ans+=1/250


            if time_left==1: #terminal takes account of exit spread
               
                ans+=-(1-lamb)*distance

            if -distance>=ub:
                
                ans+=-(1-lamb)*distance
              

            elif -distance<lb:
               
                ans+=-(1-lamb)*distance
               

    return ans 

def done_oracle(observation,cutoff=50,lb=-2,ub=6):

    distance=observation.distance
    inventory=observation.inventory
    orders_out=observation.orders_out
    time_left=observation.time_left

    if inventory==0:
        done=1
        why=f'inventory {inventory}=0'
    elif time_left>=1:
        done=1
        why=f'time up {time_left}'
    elif inventory>1:
        done=1
        why=f'inventory {inventory}>1'
    elif -distance>=ub:
        done=1
        why=f'-distance {distance} >ub {ub}'
    elif -distance<lb: 
        done=1
        why=f'-distance {distance}<lb {lb}'

    else:
        done=0 
        why=None
    return done,why

In [13]:
trader_pref_kwargs={'qty_min':-5,'qty_max':5,'sigma_pv':1}
timer_kwargs={'start':0,'end':6000,'step':1}
price_sequence_kwargs={'kappa':0.0002,'mean':100,'sigma':1,'block_length':10}
noise_kwargs={'sigma':1}
messenger_kwargs={'logging':True}
env_kwargs={'trader_arrival_rate':1,'recording':True,#'process_verbose':False,
                'bookkeep_verbose':False, 'lob_verbose':False}
sigma_n=5

def cont_coef():
    return np.random.uniform(0.2,0.8)

def personal_memory():
    return int(np.random.uniform(5,15))

trader_kwargs={'ZIP':{'prefix':'ZIP','number':10,'object_name':'WW_Zip',
                          'setup_kwargs':
                                {'market_make':True,'prior':(100,sigma_n)}},
               'HBL':{'prefix':'HBL','number':10,'object_name':'HBL',
                          'setup_kwargs':
                              {'memory':100,'grace_period':20}},
               'CON':{'prefix':'CON','number':10,'object_name':'ContTrader',
                      'setup_kwargs':
                          {'cont_coeff':cont_coef,'personal_memory':personal_memory,'profit_target':4, 'market_make':True,
                          'prior':(100,sigma_n)}},
               'NOI':{'prefix':'NOI','number':10,'object_name':'NoiseTrader',
                          'setup_kwargs':{'memory':20}}
              }

lobenv_kwargs={'cutoff':100,'profit_target':10,'loss_limit':-1,'reward_func':reward_oracle,'lamb':0.5}
agent_kwargs={'CVAE':True,'Q_H1Size':16,'Q_H2Size':16,
                   'doneModel':None,'rewardModel':None,'loss_func':None}


In [14]:
dyna_config={
	"double_q_model": False,
	"batch_size": 64,
	"learning_rate": 5e-3,
	"exploration": {
		"type": "exponential",
		"init_epsilon": 0.8,
		"min_epsilon": 0.05,
		"decay_steps": 100000,
		"decay_eps": 0.99,
        "choice":'least_bonus'
	},
	"memory": {
		"memory_capacity": 1000000,
		"prioritized": False,
        "tabular memory":True,
        
	},
	"discount": 0.99,
	"target_update_freq": 50,
	"first_update": 200,
	"modify_reward": False,
    "learn":'Q',
    'double_q_model':True,
    'model_update_freq':5,
    'planning_freq':5,
    'model':'CVAE'
    
}

In [15]:
experiment=Experiment(trader_pref_kwargs=trader_pref_kwargs,timer_kwargs=timer_kwargs,
           price_sequence_kwargs=price_sequence_kwargs,noise_kwargs=noise_kwargs,
           messenger_kwargs=messenger_kwargs,env_kwargs=env_kwargs,trader_kwargs=trader_kwargs,
           lobenv_kwargs=lobenv_kwargs,agent_kwargs=agent_kwargs,visdom=vis,dyna_kwargs=dyna_config,name=name)

sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
Device in use is  cuda
setup tabular memory


In [16]:
experiment.new_train_setup(MaxEpisodes=1000,planning_steps=1,lookback=30,thresh=3,planning=True,graph=True)

In [ ]:
experiment.train(MaxEpisodes=10001,start_episode=experiment.episode,folder='Results/'+experiment.name)

Planning is True, double Q model is True, tabular memory is True
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 21 | timestep: 27 | Ep_r:  0.08714251426115849 Profit: -1 Avg loss:-0.3008563378724598
Dyna-Q - EXP: 1 | Ep: 41 | timestep: 20 | Ep_r:  -0.9348806723206442 Profit: -2 Avg loss:-0.8665220310235966
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 61 | timestep: 10 | Ep_r:  0.034247169996478206 Profit: -1 Avg loss:-0.19711409818803904
Saving best checkpoint at episode 67 with reward 0
not strict double Q model
making new directory Results/Experiment1
Dyna-Q - EXP: 1 | Ep: 81 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.11936300736727175
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 101 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.0
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
sequence made
adding exchange to RL trader  RL
adding exchange to 

copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 981 | timestep: 100 | Ep_r:  0.7486668261738709 Profit: -2 Avg loss:0.23345516701240604
Dyna-Q - EXP: 1 | Ep: 1001 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.10716869446066296
Saving checkpoint at episode 1000
not strict double Q model
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 1021 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.10879818522677105
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 1041 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.1757784499014244
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 1061 | timestep: 2 | Ep_r:  0.0 Profit: -1 Avg loss:-0.13124121080531956
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 1081 | timestep: 3 | Ep_r:  0.0 Profit: -1 Avg loss:-0.11474121080531959
copying eval net to target

Saving best checkpoint at episode 1973 with reward 0.5395458164849399
not strict double Q model
Dyna-Q - EXP: 1 | Ep: 1981 | timestep: 2 | Ep_r:  0.0 Profit: -1 Avg loss:0.6104592858630503
Dyna-Q - EXP: 1 | Ep: 2001 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.23941291526280184
Saving checkpoint at episode 2000
not strict double Q model
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 2021 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.3925182873171618
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 2041 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.19159700505309926
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 2061 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:-0.0018248028511995758
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 2081 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg lo

adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 2981 | timestep: 2 | Ep_r:  0.0 Profit: -3 Avg loss:0.06262003470588907
Dyna-Q - EXP: 1 | Ep: 3001 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.03524117481075668
Saving checkpoint at episode 3000
not strict double Q model
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 3021 | timestep: 1 | Ep_r:  0.0 Profit: -5 Avg loss:0.4431121748002037
Dyna-Q - EXP: 1 | Ep: 3041 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:0.29451619892118797
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 3061 | timestep: 1 | Ep_r:  0.0 Profit: -8 Avg loss:0.06724551920133333
Dyna-Q - EXP: 1 | Ep: 3081 | timestep: 2 | Ep_r:  0.0 Profit: -3 Avg loss:0.11027890836441362
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
copying eval net to target net
Dyna-Q - EX

adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 3961 | timestep: 4 | Ep_r:  0.011761596 Profit: -1 Avg loss:0.2991504154887298
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 3981 | timestep: 1 | Ep_r:  0.0 Profit: -1 Avg loss:-0.05935584355367304
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 4001 | timestep: 12 | Ep_r:  0.037486051313548284 Profit: -3 Avg loss:0.2742020487102569
Saving checkpoint at episode 4000
not strict double Q model
copying eval net to target net
Dyna-Q - EXP: 1 | Ep: 4021 | timestep: 13 | Ep_r:  2.030041590800413 Profit: 1 Avg loss:0.42314724411185434
Dyna-Q - EXP: 1 | Ep: 4041 | timestep: 13 | Ep_r:  0.995 Profit: 0 Avg loss:0.28753283261703966
copying eval net to target net
sequence made
adding exchange to RL trader  RL
adding exchange to RL trader  RL
Dyna-Q - EXP: 1 | Ep: 4061 | timestep: 11 | Ep_r:  0.03786469829651342 Profit: -1 Avg loss:0.28694400164120587
Dyna-Q - EXP: 1 | Ep: 4081 | tim